In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [9]:
df = pd.read_csv('/export/scratch2/constellation-data/malafaia/ANNOVAR/files_clinvar/output.hg38_multianno.csv')
print(df.shape)
df.head()

(3706, 178)


,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,SIFT_score,SIFT_converted_rankscore,SIFT_pred,SIFT4G_score,SIFT4G_converted_rankscore,SIFT4G_pred,Polyphen2_HDIV_score,Polyphen2_HDIV_rankscore,Polyphen2_HDIV_pred,Polyphen2_HVAR_score,Polyphen2_HVAR_rankscore,Polyphen2_HVAR_pred,LRT_score,LRT_converted_rankscore,LRT_pred,LRT_Omega,MutationTaster_score,MutationTaster_converted_rankscore,MutationTaster_pred,MutationAssessor_score,MutationAssessor_rankscore,MutationAssessor_pred,FATHMM_score,FATHMM_converted_rankscore,FATHMM_pred,PROVEAN_score,PROVEAN_converted_rankscore,PROVEAN_pred,VEST4_score,VEST4_rankscore,MetaSVM_score,MetaSVM_rankscore,MetaSVM_pred,MetaLR_score,MetaLR_rankscore,MetaLR_pred,Reliability_index,MetaRNN_score,MetaRNN_rankscore,MetaRNN_pred,M-CAP_score,M-CAP_rankscore,M-CAP_pred,REVEL_score,REVEL_rankscore,MutPred_score,MutPred_rankscore,MVP_score,MVP_rankscore,gMVP_score,gMVP_rankscore,MPC_score,MPC_rankscore,PrimateAI_score,PrimateAI_rankscore,PrimateAI_pred,DEOGEN2_score,DEOGEN2_rankscore,DEOGEN2_pred,BayesDel_addAF_score,BayesDel_addAF_rankscore,BayesDel_addAF_pred,BayesDel_noAF_score,BayesDel_noAF_rankscore,BayesDel_noAF_pred,ClinPred_score,ClinPred_rankscore,ClinPred_pred,LIST-S2_score,LIST-S2_rankscore,LIST-S2_pred,VARITY_R_score,VARITY_R_rankscore,VARITY_ER_score,VARITY_ER_rankscore,VARITY_R_LOO_score,VARITY_R_LOO_rankscore,VARITY_ER_LOO_score,VARITY_ER_LOO_rankscore,ESM1b_score,ESM1b_rankscore,ESM1b_pred,EVE_score,EVE_rankscore,AlphaMissense_score,AlphaMissense_rankscore,AlphaMissense_pred,Aloft_pred,Aloft_Confidence,CADD_raw,CADD_raw_rankscore,CADD_phred,DANN_score,DANN_rankscore,fathmm-MKL_coding_score,fathmm-MKL_coding_rankscore,fathmm-MKL_coding_pred,fathmm-MKL_coding_group,fathmm-XF_coding_score,fathmm-XF_coding_rankscore,fathmm-XF_coding_pred,Eigen-raw_coding,Eigen-raw_coding_rankscore,Eigen-phred_coding,Eigen-PC-raw_coding,Eigen-PC-raw_coding_rankscore,Eigen-PC-phred_coding,GenoCanyon_score,GenoCanyon_rankscore,integrated_fitCons_score,integrated_fitCons_rankscore,integrated_confidence_value,GM12878_fitCons_score,GM12878_fitCons_rankscore,GM12878_confidence_value,H1-hESC_fitCons_score,H1-hESC_fitCons_rankscore,H1-hESC_confidence_value,HUVEC_fitCons_score,HUVEC_fitCons_rankscore,HUVEC_confidence_value,LINSIGHT,LINSIGHT_rankscore,GERP++_NR,GERP++_RS,GERP++_RS_rankscore,phyloP100way_vertebrate,phyloP100way_vertebrate_rankscore,phyloP470way_mammalian,phyloP470way_mammalian_rankscore,phyloP17way_primate,phyloP17way_primate_rankscore,phastCons100way_vertebrate,phastCons100way_vertebrate_rankscore,phastCons470way_mammalian,phastCons470way_mammalian_rankscore,phastCons17way_primate,phastCons17way_primate_rankscore,SiPhy_29way_pi,SiPhy_29way_logOdds,SiPhy_29way_logOdds_rankscore,bStatistic,bStatistic_converted_rankscore,Interpro_domain,GTEx_V8_eQTL_gene,GTEx_V8_eQTL_tissue,GTEx_V8_sQTL_gene,GTEx_V8_sQTL_tissue,eQTLGen_snp_id,1000g2015aug_all,AF,AF_popmax,AF_male,AF_female,AF_raw,AF_afr,AF_sas,AF_amr,AF_eas,AF_nfe,AF_fin,AF_asj,AF_oth,non_topmed_AF_popmax,non_neuro_AF_popmax,non_cancer_AF_popmax,controls_AF_popmax,Otherinfo1
0,19,11089263,11089263,C,G,ncRNA_exonic,LDLR-AS1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,1
1,19,11089281,11089281,G,T,ncRNA_exonic,LDLR-AS1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,0.00898562,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,0
2,19,11089309,11089311,CAG,C,ncRNA_exonic,LDLR-AS1,.,

In [8]:
for i in df.columns: print(i)

Chr
Start
End
Ref
Alt
Func.refGene
Gene.refGene
GeneDetail.refGene
ExonicFunc.refGene
AAChange.refGene
SIFT_score
SIFT_converted_rankscore
SIFT_pred
SIFT4G_score
SIFT4G_converted_rankscore
SIFT4G_pred
Polyphen2_HDIV_score
Polyphen2_HDIV_rankscore
Polyphen2_HDIV_pred
Polyphen2_HVAR_score
Polyphen2_HVAR_rankscore
Polyphen2_HVAR_pred
LRT_score
LRT_converted_rankscore
LRT_pred
LRT_Omega
MutationTaster_score
MutationTaster_converted_rankscore
MutationTaster_pred
MutationAssessor_score
MutationAssessor_rankscore
MutationAssessor_pred
FATHMM_score
FATHMM_converted_rankscore
FATHMM_pred
PROVEAN_score
PROVEAN_converted_rankscore
PROVEAN_pred
VEST4_score
VEST4_rankscore
MetaSVM_score
MetaSVM_rankscore
MetaSVM_pred
MetaLR_score
MetaLR_rankscore
MetaLR_pred
Reliability_index
MetaRNN_score
MetaRNN_rankscore
MetaRNN_pred
M-CAP_score
M-CAP_rankscore
M-CAP_pred
REVEL_score
REVEL_rankscore
MutPred_score
MutPred_rankscore
MVP_score
MVP_rankscore
gMVP_score
gMVP_rankscore
MPC_score
MPC_rankscore
PrimateA

In [14]:
cols_input = ['Chr', 'Start', 'End', 'Ref', 'Alt']
cols_other = ['Func.refGene', 'Gene.refGene']
cols_af = df.filter(regex='AF').columns.tolist()
cols_1kgp = df.filter(regex='1000g').columns.tolist()
cols_gnomad = df.filter(regex='gnomad').columns.tolist() # nothing
cols_revel = df.filter(regex='REVEL').columns.tolist()
cols_metarnn = df.filter(regex='MetaRNN').columns.tolist()
cols_sift = df.filter(regex='SIFT_').columns.tolist()
cols_mtaster = df.filter(regex='MutationTaster').columns.tolist()
cols_provean = df.filter(regex='Provean').columns.tolist()

cols = cols_input + cols_other + cols_af + cols_1kgp + cols_gnomad + cols_revel + cols_metarnn + cols_sift + cols_mtaster + cols_provean
df_useful = df[cols]
print(df_useful.shape)
df_useful.head()

(3706, 42)


,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,BayesDel_addAF_score,BayesDel_addAF_rankscore,BayesDel_addAF_pred,BayesDel_noAF_score,BayesDel_noAF_rankscore,BayesDel_noAF_pred,AF,AF_popmax,AF_male,AF_female,AF_raw,AF_afr,AF_sas,AF_amr,AF_eas,AF_nfe,AF_fin,AF_asj,AF_oth,non_topmed_AF_popmax,non_neuro_AF_popmax,non_cancer_AF_popmax,controls_AF_popmax,1000g2015aug_all,REVEL_score,REVEL_rankscore,MetaRNN_score,MetaRNN_rankscore,MetaRNN_pred,SIFT_score,SIFT_converted_rankscore,SIFT_pred,MutationTaster_score,MutationTaster_converted_rankscore,MutationTaster_pred
0,19,11089263,11089263,C,G,ncRNA_exonic,LDLR-AS1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
1,19,11089281,11089281,G,T,ncRNA_exonic,LDLR-AS1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,0.00898562,.,.,.,.,.,.,.,.,.,.,.
2,19,11089309,11089311,CAG,C,ncRNA_exonic,LDLR-AS1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
3,19,11089318,11089319,AC,A,ncRNA_exonic,LDLR-AS1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
4,19,11089318,11089458,ACGGGTTAAAAAGCCGATGTCACATCGGCCGTTCGAAACTCCTCCT...,A,ncRNA_exonic,LDLR-AS1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.


In [19]:
for i in df_useful.columns:
    print(f"{i}: {df_useful[i].loc[df_useful[i]=='.'].shape[0]}")

Chr: 0
Start: 0
End: 0
Ref: 0
Alt: 0
Func.refGene: 229
Gene.refGene: 229
BayesDel_addAF_score: 1856
BayesDel_addAF_rankscore: 1856
BayesDel_addAF_pred: 1856
BayesDel_noAF_score: 1856
BayesDel_noAF_rankscore: 1856
BayesDel_noAF_pred: 1856
AF: 2923
AF_popmax: 2951
AF_male: 2923
AF_female: 2923
AF_raw: 2923
AF_afr: 2923
AF_sas: 2923
AF_amr: 2923
AF_eas: 2923
AF_nfe: 2923
AF_fin: 2923
AF_asj: 2923
AF_oth: 2923
non_topmed_AF_popmax: 2961
non_neuro_AF_popmax: 2982
non_cancer_AF_popmax: 2969
controls_AF_popmax: 3202
1000g2015aug_all: 3500
REVEL_score: 2225
REVEL_rankscore: 2225
MetaRNN_score: 2171
MetaRNN_rankscore: 2171
MetaRNN_pred: 2171
SIFT_score: 2173
SIFT_converted_rankscore: 2173
SIFT_pred: 2173
MutationTaster_score: 1856
MutationTaster_converted_rankscore: 1856
MutationTaster_pred: 1856
